# Prompt Template
## template를 만드는 방법: 

```python
t = PromptTemplate.from_template("What is the capital if {country}.")
t.format(country="France")

```

- {} 는 placeholder  
- 템플릿을 만들면 좋은점은 validation 유효성 검사를 할 수 있다는 점.
  - placeholder를 작성하지 않으면 에러가 발생한다.
- prompt template 를 디스크에 저장하고 load 할 수 있기 때문. 규모가 큰 language model 을 만들기 시작할 때 prompt는 정말 중요하다.

### .from_template 를 사용하지 않는 경우
```python
t = PromptTemplate(
  template="What is the capital if {country}.",
  input_variables=["country"],
)
t.format(country="France")
```
- from_template만 쓰면 되는 것과 같이 동작한다.
- from_template를 사용하면 자동으로 랭체인이 어떤 변수가 우리가 필요한지 알아서 계산하여 유효성 검사를 해주는 것.


In [19]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate, FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],

)
# {} 는 placeholder
t = PromptTemplate.from_template("What is the capital if {country}.")
t.format(country="France")

'What is the capital if France.'

# FewshotPromptTemplate 와 Fewshot Learning
## FewShotPromptTemplate가 하는 일
Fewshot은 모델에게 예제들을 준다는 뜻과 같다.
더 나은 대답을 할 수 있도록 하는 예제들을 더 줄 수 있다.
예를들어 모델에게 콤마를 써서 구분해줘, 소문자만 써야해 등 prompt로 전달하는 것보다 내가 원하는 것을 예제로 보여주는 것이 더 성공적일 것이다.
왜냐하면? 모델이 텍스트를 만들기 때문.
이를 통해 예제를 형식화할 수 있다.

대화 기록 같은 것들을 데이터베이스에서 가져와서 FewShotPromptTemplate을 사용하여 형식화 시켜주면 더 빠르게 잘 만들 수 있다.

예제들을 리스트로 만들어 본다. example

## FewShotPromptTemplate 사용
1.  예제의 형식을 지정 
  - 형식화 하려면 형식 지정 도구를 만들어야 한다.
  - 예제는 데이터베이스에서 가져오거나 할 수 있다.

  ```python
    # 예제의 형식을 지정하는 방법
  # Human 에는 질문을 넣어줄거고, AI에는 답변을 넣어준다.
  example_template = """
      Human:{question}
      AI:{answer}
  """
  ```
  
2.  FewShotPromptTemplate을 사용한 실제 prompt

suffix: 형식화 된 모든 예제 마지막에 나오는 내용.
```python
prompt = FewShotPromptWithTemplates(
    example_prompt=example_prompt,
    examples=examples,
    suffix="Human: What do you know about {country}?"
    input_variables=["country"]
)
```
- example 로 형식화 하고 suffix 로 내용을 마지막에 포함.





In [20]:
chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],

)

examples = [
    {
        "question": "What do you know about France?",
        "answer":"""
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer":"""
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer":"""
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    }
]

# 예제의 형식을 지정하는 방법
# Human 에는 질문을 넣어줄거고, AI에는 답변을 넣어준다.
example_template = """
    Human:{question}
    AI:{answer}
"""

example_prompt = PromptTemplate.from_template(example_template)

# FewShotPromptTemplate을 사용한 실제 prompt
prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"],
)

prompt.format(country="Korea")

chain = prompt | chat

chain.invoke({
    "country":"korea"
})

AI: 
        Here is what I know:
        Capital: Seoul
        Language: Korean
        Food: Kimchi and Bibimbap
        Currency: South Korean Won

AIMessageChunk(content='AI: \n        Here is what I know:\n        Capital: Seoul\n        Language: Korean\n        Food: Kimchi and Bibimbap\n        Currency: South Korean Won')

## FewShotChatMessagePromptTemplate
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.prompts import ChatPromptTemplate

FewShotChatMessagePromptTemplate 는 suffix 같은거 필요 없음.


In [21]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)


In [30]:
examples = [
    {
        "country": "France",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "country": "Italy",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "country": "Greece",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]


example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "What do you know about {country}?"),
        ("ai", "{answer}"),
    ]
)

# example 목록에 있는 각 예제의 형식을 지정.
# ChatPromptTemplate를 사용하여 형식을 지정해줌.
example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a geography expert, you give short answers and short opinion of country."),
        example_prompt,
        ("human", "What do you know about {country}?"),
    ]
)


chain = final_prompt | chat

chain.invoke({"country": "korea"})

I know this:
Capital: Seoul
Language: Korean
Food: Kimchi and Bibimbap
Currency: South Korean Won

AIMessageChunk(content='I know this:\nCapital: Seoul\nLanguage: Korean\nFood: Kimchi and Bibimbap\nCurrency: South Korean Won')